# Chapter 9 Up and Running with TensorFlow

## Installation

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.9.0


## Creating Your First Graph and Running It in a Session

In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y +2

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [5]:
sess.close()

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [7]:
print(result)

42


In [8]:
sess.close()

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [10]:
print(result)

42


In [11]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [12]:
sess.close()

## Managing Graphs

## Lifecycle of a Node Value

In [13]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [14]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with TensorFlow

In [15]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [16]:
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [17]:
import tensorflow as tf
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [18]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

## Implementing Gradient Descent

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [20]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
# gradients = 2/m * tf.matmul(tf.transpose(X), error)

# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 11.991551
Epoch 100 MSE = 0.5447721
Epoch 200 MSE = 0.52546453
Epoch 300 MSE = 0.52444583
Epoch 400 MSE = 0.52433705
Epoch 500 MSE = 0.5243228
Epoch 600 MSE = 0.52432096
Epoch 700 MSE = 0.52432084
Epoch 800 MSE = 0.5243212
Epoch 900 MSE = 0.5243208


In [21]:
best_theta

array([[ 2.0685582 ],
       [ 0.82963014],
       [ 0.11875364],
       [-0.2655474 ],
       [ 0.30571318],
       [-0.00450241],
       [-0.03932667],
       [-0.89986086],
       [-0.87051755]], dtype=float32)

## Feeding Data to the Training Algorithm

## Saving and Restoring Models

## Visualizing the Graph and Training Curves Using TensorBoard

In [24]:
tf.reset_default_graph()

In [29]:
A = tf.placeholder(tf.float32, shape=(None))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [1, 2, 3]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
print(B_val_1)
print(B_val_2)

[6. 7. 8.]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [66]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

n_epochs = 10
learning_rate = 0.01

theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

# saver = tf.train.Saver()

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}".format(root_logdir, now)
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [67]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    # save_path = saver.save(sess, "/tmp/my_model.ckpt")

In [68]:
file_writer.close()
best_theta

array([[ 2.0709088 ],
       [ 0.8432678 ],
       [ 0.12913173],
       [-0.30181286],
       [ 0.3520365 ],
       [-0.00591041],
       [-0.08734914],
       [-0.84622973],
       [-0.81247276]], dtype=float32)

In [74]:
tf.reset_default_graph()

## Name Scopes

## Modularity

## Sharing Variables